In [7]:
import tensorflow as tf
import os
import sys
import cv2
import numpy as np
import tensorflow.contrib.slim as slim
import tensorflow_hub as hub
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
gpu_options = tf.GPUOptions(allow_growth=True)
config = tf.ConfigProto(gpu_options=gpu_options,log_device_placement=True,allow_soft_placement=True)
import lmdb
%matplotlib inline
import matplotlib.pyplot as plt
import pickle
import pycocotools.mask as maskUtils

sys.path.append('../utilities/')
if 'utils_data_gen' in sys.modules:
    del sys.modules['utils_data_gen']
from utils_data_gen import *

if 'utils_evaluation' in sys.modules:
    del sys.modules['utils_evaluation']
from utils_evaluation import *

if 'utils_model' in sys.modules:
    del sys.modules['utils_model']
from utils_model import *

print('Import done')

Import done


In [2]:
cloud = 0
print(tf.contrib.eager.num_gpus())

1


# Data Path

In [3]:
if cloud ==1:
    os.environ["TFHUB_CACHE_DIR"] = "/home/sonamgoenka/tfhub_modules"
    slim_models_path = '/home/david_hc95/grounding/iccv19_grounding/models/'
    sys.path.append(slim_models_path)
    data_path = '/home/ha2436/data/'
else:
    os.environ["TFHUB_CACHE_DIR"] = "/home/sonam/tfhub_modules"
    slim_models_path = '/home/sonam/models/'
    sys.path.append(slim_models_path)
    data_path = '/dvmm-filer2/datasets/Groundings/data/'
    

In [4]:
# Coco sentences has sentence, segmentation, Vg has query
# MS COCO
data_path_coco = data_path + 'mscoco/'
dict_paths_coco = [data_path_coco + 'train2014.pickle',
                  data_path_coco + 'val2014.pickle']
lmdb_path_coco = data_path_coco + 'MSCOCO_jpg.lmdb'

# Flickr It
data_path_flickr = data_path + 'flickr30k/'
dict_paths_flickr = [data_path_flickr + 'flickr30k_val.pickle',
                  data_path_flickr + 'flickr30k_test.pickle']
#lmdb_path_refer = data_path_refer + 'MSCOCO_jpg.lmdb' ?


# Visual Genome
data_path_vg  = data_path + 'visualgenome/'
dict_paths_vg = [data_path_vg + 'train.pickle',
                 data_path_vg + 'val2014.pickle',
                data_path_vg + 'test.pickle']

lmdb_path_vg = data_path_vg + 'data.lmdb'

# Refer It

ref_path = data_path + 'referit/referit_splits/'
lmdb_path_ref =  data_path + 'referit/refclef_data/all_data.lmdb'

ref_dict_path = ref_path + 'test.pickle' 


In [5]:
#Loading mscoco data
print('Loading Mscoco...')
with open(dict_paths_coco[0], 'rb') as f:
    dict_train_coco = pickle.load(f, encoding='latin1')
    ids_train_coco = list(dict_train_coco.keys())   
with open(dict_paths_coco[1], 'rb') as f:
    dict_val_coco = pickle.load(f, encoding='latin1')
    ids_val_coco = list(dict_val_coco.keys())
lmdb_env_coco = lmdb.open(lmdb_path_coco, map_size=int(1e11), readonly=True, lock=False)
txn_coco = lmdb_env_coco.begin(write=False)

#Loading Flickr, dict contains image
print('Loading Flickr30k...')
with open(dict_paths_flickr[1], 'rb') as f:
    dict_test_flickr = pickle.load(f, encoding='latin1')
    
#loading VG
print('Loading VG...')
with open(dict_paths_vg[0], 'rb') as f:
    dict_train_vg = pickle.load(f, encoding='latin1')
    ids_train_vg = list(dict_train_vg.keys())
    
with open(dict_paths_vg[2], 'rb') as f:
    dict_test_vg = pickle.load(f, encoding='latin1')
    ids_test_vg = list(dict_test_vg.keys())


lmdb_env_vg = lmdb.open(lmdb_path_vg, map_size=int(1e11), readonly=True, lock=False)
txn_vg = lmdb_env_vg.begin(write=False)

#loading Refer
print('Loading Refer...')
with open(ref_dict_path, 'rb') as f:
    dict_test_ref = pickle.load(f, encoding='latin1')
    ids_test_ref = list(dict_test_ref.keys())
    
lmdb_env_ref = lmdb.open(lmdb_path_ref, map_size=int(1e11), readonly=True, lock=False)
txn_ref = lmdb_env_ref.begin(write=False)

print('Loading Done')

Loading Mscoco...
Loading Flickr30k...
Loading VG...
Loading Refer...
Loading Done


In [ ]:
#print(list(dict_train_vg.keys())[:5])
#dict_train_vg['2344382']
#print(list(dict_train_coco.keys())[:5])
#dict_train_coco['329242']

# Params

In [10]:
# train on which dataset
dict_train = dict_train_vg
ids_train = ids_train_vg
txn = txn_vg

n_batch = 4
gamma_1 = 5.0
gamma_2 = 10.0
n_iter_per_epoch = 2  #int(len(dict_train)/n_batch)
n_iter_per_epoch_val = 2  #int(5000/n_batch)
n_epochs = 1 #N of epochs
reg_val = .0005
# visual params
lr_value = .001
log_file = './logs/vgg/vgg'

# Architecture
#Temp :'Phrase_Grounding_NewAttn_VG_Seg_COCO_Pyramid_VGG'
model_name =  './Model_' + 'Phrase_Grounding_NewAttn_VG_VGG'
conv_kernel_size = 3 #1
n_layers = 1 #3
vis_method = depth_selection_newattn_vgg # Pyramid
segment_branch = 0
msk_size = 38 #38 for pnasnet, 37 for vgg
visual_model = 'vgg' #pnasnet, vgg
use_new_attn = False
latent_space_dim = 512

# Built Model

In [12]:
tf.reset_default_graph()
try:
    sess.close()
except:
    print('Initialising for first time')
    
sess = tf.InteractiveSession(config=config)
mode = tf.placeholder(tf.string, name='mode')
isTraining = tf.equal(mode, 'train')
regularizer = tf.contrib.layers.l2_regularizer(reg_val)

with tf.device('/gpu:0'):
    
    #Building visual model
    print('Building Visual Model...')
    input_img = tf.placeholder(tf.float32, (None,299,299,3), name='input_img')
    if visual_model == 'vgg':
        pre_processed_img = pre_process(input_img, 'vgg_preprocessing')
        vis_model = pre_trained_load(model_name='vgg_16', image_shape=(None,299,299,3),
                                  input_tensor=pre_processed_img, session=sess, is_training=False, global_pool=False)
    if visual_model == 'pnasnet':
        pre_processed_img = pre_process(input_img, 'inception_preprocessing')
        vis_model = pre_trained_load(model_name='pnasnet_large', image_shape=(None,299,299,3),
                                 input_tensor=pre_processed_img, session=sess, is_training=False, global_pool=True)
        


        # Type of Depth Selection
    v = vis_method(vis_model,regularizer, conv_kernel_size, n_layers) #(?,1225,4,1024)
    
    #building text model
    print('Building Text Model...')
    text_batch = tf.placeholder('string', shape=[None], name='text_input')
    elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)
    embeddings = elmo(text_batch, signature="default", as_dict=True)
    lstm1_embd = embeddings['lstm_outputs1'] #?xTXD
    lstm2_embd = embeddings['lstm_outputs2'] #?xTXD
    
    
    if segment_branch == 1:
        seg_mask_in = tf.placeholder(tf.float32, (None,None,msk_size,msk_size), name='seg_mask')
        is_seg = tf.placeholder(tf.int32,(1) ,name='seg_vs_ground_train')
        is_seg = tf.reshape(is_seg, [])
        w_embedding  = tf.cond(tf.equal(is_seg, 1), lambda: tf.identity(embeddings['elmo'], name='elmo_word'), lambda: tf.identity(embeddings['elmo'], name='elmo_word_embd'))
    else:
        w_embedding = tf.identity(embeddings['elmo'], name='elmo_word_embd') #?xTXD
    
    idx = embeddings['sequence_len']-1
    # generates first dimension batch sixe, other length 
    batch_idx = tf.stack([tf.range(0,tf.size(idx),1),idx],axis=1)

    dim = lstm1_embd.get_shape().as_list()[-1]
    
    # Concatenate first of backward with last of forward to get sentence embeddings
    sen_lstm_1 = tf.concat([lstm1_embd[:,0,int(dim/2):],
                            tf.gather_nd(lstm1_embd[:,:,:int(dim/2)],batch_idx)], axis=-1) #[batch,dim]
    sen_lstm_2 = tf.concat([lstm2_embd[:,0,int(dim/2):],
                            tf.gather_nd(lstm2_embd[:,:,:int(dim/2)],batch_idx)], axis=-1) #[batch,dim]
    sen_lstm_both = tf.concat([tf.expand_dims(sen_lstm_1,axis=2),
                               tf.expand_dims(sen_lstm_2,axis=2)], axis=2, name='elmo_sen_embd') #[batch,dim,2]
    
    sen_embedding = tf.layers.dense(sen_lstm_both, units=1,use_bias=False) #?xDx1
    sen_embedding = tf.squeeze(sen_embedding,axis=2) 
    sen_embedding = tf.layers.dense(sen_embedding, units=latent_space_dim) #batch*dim
    sen_embedding = tf.nn.leaky_relu(sen_embedding,alpha=.25)
    sen_embedding = tf.layers.dense(sen_embedding, units=latent_space_dim)
    sen_embedding = tf.nn.leaky_relu(sen_embedding,alpha=.25)
    sen_embedding = tf.nn.l2_normalize(sen_embedding, axis=-1, name='sen_embedding')
        
    w_embedding = tf.layers.dense(w_embedding, units=latent_space_dim)
    w_embedding = tf.nn.leaky_relu(w_embedding,alpha=.25)
    w_embedding = tf.layers.dense(w_embedding, units=latent_space_dim)
    w_embedding = tf.nn.leaky_relu(w_embedding,alpha=.25)
    w_embedding = tf.nn.l2_normalize(w_embedding, axis=-1, name='w_embedding')
    
   
    print('Generating Heatmaps')
    if use_new_attn:
        heatmap_w,heatmap_s = attn_new(w_embedding,v,sen_embedding)
    else:
        heatmap_w,heatmap_s,R_i,R_s = attn(w_embedding,v,sen_embedding)
    
    
    if segment_branch == 1:
        with tf.variable_scope('segmentation'):
            if use_new_attn:
                heatmap_w, _, _ ,_  = attn_new(w_embedding,v,sen_embedding)
            else:
                h_w, _, score_w, _ = attn(w_embedding,v,sen_embedding)
            h_w = tf.reshape(h_w,[tf.shape(h_w)[0],tf.shape(h_w)[1],h_w.shape[2]*h_w.shape[3]])
            #flatten
            seg_mask_r = tf.reshape(seg_mask_in,[tf.shape(seg_mask_in)[0],tf.shape(seg_mask_in)[1],
                                        seg_mask_in.shape[2]*seg_mask_in.shape[3]])
            seg_mask_r = seg_mask_r[:,:tf.shape(h_w)[1],:]
            
        loss_seg = seg_loss(h_w, seg_mask_r) + tf.losses.get_regularization_loss()
        
if use_new_attn:   
    loss_grnd = attn_loss_new(w_embedding,v,sen_embedding, gamma_1, gamma_2) + tf.losses.get_regularization_loss()
else:
    loss_grnd = attn_loss(w_embedding,v,sen_embedding, gamma_1, gamma_2) + tf.losses.get_regularization_loss()
    
lr = tf.placeholder(tf.float32, shape=[], name='learning_rate')
opt = tf.train.AdamOptimizer(lr)
train_vars = list(set(tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)) -  set(vis_model.model_weights))
train_op_grnd = opt.minimize(loss_grnd, var_list=train_vars)

if segment_branch == 1:
    train_op_seg = opt.minimize(loss_seg, var_list=train_vars)

global_saver = tf.train.Saver()
train_writer = tf.summary.FileWriter(log_file, sess.graph)
print('Model Built')

Building Visual Model...
Instructions for updating:
Use keras.layers.conv2d instead.


W0504 14:34:57.992147 140263084984128 deprecation.py:323] From ../utilities/utils_model.py:251: conv2d (from tensorflow.python.layers.convolutional) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.conv2d instead.


Building Text Model...
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0504 14:34:59.094756 140263084984128 saver.py:1483] Saver not created because there are no variables in the graph to restore


Generating Heatmaps
Instructions for updating:
Use tf.cast instead.


W0504 14:34:59.890670 140263084984128 deprecation.py:323] From /home/sonam/anaconda3/envs/tf_gpu_1_13/lib/python3.6/site-packages/tensorflow/python/ops/math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.


Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


W0504 14:34:59.907364 140263084984128 deprecation.py:323] From /home/sonam/anaconda3/envs/tf_gpu_1_13/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:102: div (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


Model Built


# Batch Gen

In [13]:
valid_ids_train, train_images = preload_images(ids_train[:100], txn)    
print('Done pre-processing training images')
print('train images before',len(ids_train), 'after', len(valid_ids_train))
if segment_branch == 1:
    valid_ids_train_coco, train_images_coco = preload_images(ids_train_coco[:100], txn_coco) 
print('done')

Processed:  0  images
Finished preprocessing images.
Done pre-processing training images
train images before 77398 after 98
done


# Train

In [14]:
# print('Initializing...')
_ = sess.run([tf.global_variables_initializer(), tf.tables_initializer()])
#loading pretrained inception weights
print('Loading visual path model (InceptionV4)...')
vis_model.load_weights()
print('Model Loaded')

Loading visual path model (InceptionV4)...
Instructions for updating:
Use standard file APIs to check for files with this prefix.


W0504 14:35:15.530646 140263084984128 deprecation.py:323] From /home/sonam/anaconda3/envs/tf_gpu_1_13/lib/python3.6/site-packages/tensorflow/python/training/saver.py:1266: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


INFO:tensorflow:Restoring parameters from /home/sonam/models/vgg_16.ckpt


I0504 14:35:15.537753 140263084984128 saver.py:1270] Restoring parameters from /home/sonam/models/vgg_16.ckpt


Model Loaded


In [ ]:
print('Start training...')
validation_loss = np.zeros((n_epochs,))
train_loss = np.zeros((n_epochs,))
train_grnd_loss = np.zeros((n_epochs,))
train_seg_loss = np.zeros((n_epochs,))
val_grnd_loss = np.zeros((n_epochs,))
val_seg_loss = np.zeros((n_epochs,))


for e in range(n_epochs):
    print('\n\n=====Epoch: %d'%e)
    #train phase
    avg_loss = [0,0]
    
    if e < 9:
        lr_value = lr_value
    elif 9 <= e < 14:
        lr_value = lr_value / 2.0
    elif e >= 14:
        lr_value = lr_value/ 4.0 
   
    print('===Train')
    
    if segment_branch == 1:
        for i in range(n_iter_per_epoch):
            
            img_batch, cap_batch = batch_gen(valid_ids_train, dict_train, train_images, n_batch)
            cat_batch, msk_batch  = batch_gen_seg_add_cat_map(valid_ids_train_coco, dict_train_coco, train_images_coco, n_batch, msk_size)
            feed_dict = {input_img: img_batch, text_batch: cap_batch, mode: 'train', lr: lr_value,is_seg: 0}
            loss_grnd_value, _ = sess.run([loss_grnd, train_op_grnd], feed_dict)
            feed_dict.update({text_batch: cat_batch, seg_mask_in:msk_batch,is_seg :1})
            loss_seg_value, _ = sess.run([loss_seg, train_op_seg], feed_dict) 
            avg_loss[0]+=loss_grnd_value
            avg_loss[1]+=loss_seg_value
            v = [i*n_batch,n_iter_per_epoch*n_batch,avg_loss[0]/float(i+1),avg_loss[1]/float(i+1), (avg_loss[0]+avg_loss[1])/float(i+1)]
            sys.stdout.write(f'Sample {v[0]}/{v[1]}, grnd_loss_value:{v[2]:.2f}, seg_loss_value:{v[3]:.2f}, overall_loss:{v[4]:.2f}\r')
            sys.stdout.flush()
            train_loss[e] = (avg_loss[0]+avg_loss[1])/float(n_iter_per_epoch+1)
            train_grnd_loss[e] = (avg_loss[0])/float(n_iter_per_epoch+1)
            train_seg_loss[e] = (avg_loss[1])/float(n_iter_per_epoch+1)         
    else:
        for i in range(n_iter_per_epoch):
            img_batch, cap_batch = batch_gen(valid_ids_train, dict_train, train_images, n_batch)
            feed_dict = {input_img: img_batch, text_batch: cap_batch, mode: 'train', lr: lr_value}
            loss_grnd_value, _ = sess.run([loss_grnd, train_op_grnd], feed_dict)
            avg_loss[0]+=loss_grnd_value
            v = [i*n_batch,n_iter_per_epoch*n_batch,avg_loss[0]/float(i+1),avg_loss[1]/float(i+1), (avg_loss[0]+avg_loss[1])/float(i+1)]
            sys.stdout.write(f'Sample {v[0]}/{v[1]}, grnd_loss_value:{v[2]:.2f}, seg_loss_value:{v[3]:.2f}, overall_loss:{v[4]:.2f}\r')
            sys.stdout.flush()
            train_loss[e] = (avg_loss[0]+avg_loss[1])/float(n_iter_per_epoch+1)
    
    if e == 10:
        print('saving model')
        global_saver.save(sess, model_name + str(e))
        print('model saved')
    
print('Training done.')
#saving the session
print('Saving model...')
global_saver.save(sess,  model_name + str(n_epochs))
print('Saving done.')

Start training...


=====Epoch: 0
===Train
Training done.nd_loss_value:6.53, seg_loss_value:0.00, overall_loss:6.53
Saving model...


In [ ]:
plt.plot(validation_loss, label = 'total_val')
plt.plot(train_loss, label='train_loss')
plt.legend()
plt.show()

# Evaluation

In [ ]:
tf.reset_default_graph()

In [ ]:
# If loading pretrain model run this
load_name = 'model_AttGrnd_3layerbase_pyramid15'
model_path = './Models/' + load_name
print('Loading grounding pretrained model...')
print(model_path)
sess, graph = load_model(model_path,config)
print('Model Loaded')

In [ ]:
# Getting Right Tensors
R_i = sess.graph.get_tensor_by_name("attention/score_word:0")#/transpose_2
R_s = sess.graph.get_tensor_by_name("attention/score_sentence:0")
heatmap_w = sess.graph.get_tensor_by_name("attention/heatmap_word:0")
heatmap_s = sess.graph.get_tensor_by_name("attention/heatmap_sentence:0")
lvl_idx_wrd = sess.graph.get_tensor_by_name('attention/level_index_word:0')
lvl_scr_wrd = sess.graph.get_tensor_by_name('attention/level_score_word:0')
lvl_idx_sen = sess.graph.get_tensor_by_name('attention/level_index_sentence:0')

# Flickr

In [ ]:
# Flickr
print('Test on Flickr30k\n')
num_tst = len(dict_test_flickr)
iou_acc_f30k, hit_acc_f30k, att_acc_f30k = validate_flickr30k(dict_test_flickr, num_tst,heatmap_w, R_i, R_s,sess)
print('done testing')
# Save the results

In [ ]:
print('Iou',iou_acc_f30k, 'Hit',hit_acc_f30k, 'Att', att_acc_f30k  )

# Refer It

In [ ]:
#valid_ids_test_ref, test_images_ref = pre_images(ids_test_ref, txn_ref) 

In [ ]:
num_tst_ref = len(dict_test_ref)
iou_acc_w_ref, hit_acc_w_ref, att_acc_w_ref, iou_acc_s_ref, hit_acc_s_ref, att_acc_s_ref = validate_referit(
    dict_test_ref,txn_ref,num_tst_ref, heatmap_w,heatmap_s,R_i, R_s,lvl_idx_wrd, lvl_idx_sen, lvl_scr_wrd, sess)

In [ ]:
print('Iou_w', iou_acc_w_ref,'hit_w', hit_acc_w_ref, 'att_w',att_acc_w_ref,
      'iou_s' , iou_acc_s_ref,'hit_s', hit_acc_s_ref,'att_s', att_acc_s_ref)

# Visual Genome

In [ ]:
num_tst_vg = len(dict_test_vg)
#num_tst_vg = 2
iou_acc_w_vg, hit_acc_w_vg, att_acc_w_vg,iou_acc_s_vg, hit_acc_s_vg ,att_acc_s_vg = validate_vg(
    dict_test_vg, txn_vg,num_tst_vg,heatmap_w,heatmap_s, R_i, R_s, lvl_idx_wrd, lvl_idx_sen, lvl_scr_wrd, sess)
print('Done')

In [ ]:
print('Iou_w', iou_acc_w_vg,'hit_w', hit_acc_w_vg, 'att_w',att_acc_w_vg,
      'iou_s' , iou_acc_s_vg,'hit_s', hit_acc_s_vg,'att_s', att_acc_s_vg)